In [1]:
import numpy as np
from collections import Counter
from itertools import chain
from math import floor

In [2]:
# Given a puzzle return a list of 3x3 subblocks, in order from left to right, top to bottom
def init_subblocks(puzzle):
    subblocks = [ np.hsplit(x,3) for x in np.vsplit(puzzle,3)]
    subblocks = list(chain.from_iterable(subblocks))
    return subblocks

# Given a cell (i,j) return the subblock that it resides in
def get_subblock(subblocks,i,j):
    x,y = floor(i/3), floor(j/3)
    i = 3 * x + y
    return subblocks[i]# (i,j): Coordinates of a blank value

# Given a NON blank cell (i,j), return the set of all the dependent values (values that puzzle[i][j] CANNOT be)
def get_dependent_values(subblocks,i,j):
    subblock = set(get_subblock(subblocks,i,j).flatten())
    row = set(puzzle[i,:])
    col = set(puzzle[:,j])
    # Set union of subblock, row and column values removing blanks  and the value of the current cell
    return subblock.union(row,col).difference({-1, puzzle[i][j]})

# Given a 1D collection of numbers, return the number of missing values
def _cost(values):
    return len(set(range(1,10)).difference(set(values)))

# Sum of the number of missing values over each row, column and subblock
def cost(puzzle):
    block_cost = 0
    row_cost = 0
    col_cost = 0

    x,y,z = np.hsplit(puzzle, 3)
    (x1,x2,x3), (y1,y2,y3), (z1,z2,z3) = np.vsplit(x,3), np.vsplit(y,3), np.vsplit(z,3)
    blocks = [x1,x2,x3,y1,y2,y3,z1,z2,z3]
    block_cost = sum([_cost(x.flatten()) for x in blocks])

    for row in puzzle:
        row_cost += _cost(row)
    for col in puzzle.T:
        col_cost += _cost(col)
    return block_cost + row_cost + col_cost

# Compute the empirical probabilities (probabilites of each cell taking a value over the samples)
def empirical_probabilities(samples):
    p = np.empty((9,9,9))
    N = len(samples)
    for i in range(9):
        for j in range(9):
            numbers_ij = [sample[i][j] for sample in samples]
            count = Counter(numbers_ij)
            p[i][j] = [count[i]/N for i in range(1,10)]
    return p

# Sample a probability distribution
# sample[i][j]: The value of cell (i,j)
def sample(P):
        sample = np.empty((9,9),dtype=int)
        # sample = np.empty((9,9))
        values = range(1,10)
        for i in range(9):
            for j in range(9):
                sample[i][j] = np.random.choice(values,size=None,p=P[i][j])
        return sample

# Verify that distribution sums to one for each cell and that probabilties are between 0 and 1
def check_P(P):
    sum_to_one = np.allclose(np.sum(P,axis=2), np.ones(9))
    valid_range = not(np.any(model.P < 0)) and not(np.any(model.P > 1))
    return sum_to_one and valid_range

In [3]:
class Model:
    
    def __init__(self, puzzle):
        self.puzzle = puzzle
        self.subblocks = init_subblocks(self.puzzle)
        self.P = self._init_P()
        self.P0 = self.P.copy() 
    
    # Initialise probability matrix
    # P[i][j][k]: Probability of cell (i,j) taking the value k
    def _init_P(self):
        P = np.empty((9,9,9))
        for i in range(9):
            for j in range(9):
                x = int(self.puzzle[i][j])
                if x != -1:
                    # If the cell has a value k that is already given, fix its probability to 1
                    P[i][j] = [1 if i==x else 0 for i in range(1,10)]
                else:
                    dependent_values = get_dependent_values(self.subblocks,i,j)
                    n = 9 - len(dependent_values)
                    for k in range(9):
                        v = k + 1
                        P[i][j][k] = 0 if v in dependent_values else 1/n
        return P
   
    def update(self,alpha,empirical_probabilities):
        self.P = alpha * self.P + (1-alpha) * empirical_probabilities

In [4]:
# Stopping Condition
def f(beta,P):
    return np.amin(np.amax(P,axis=2)) > beta

def main(model, N, Q, Q1, alpha, beta):
    print(f"Updating model over {N} iterations")
    print(f"Population Size={Q},Sample Size={Q1},alpha={alpha},beta={beta}")
    for n in range(N):
        if f(beta,model.P):
            print(f"Stopping condition met at iteration {n}")
            break
        samples = [ sample(model.P) for _ in range(Q)]
        samples.sort(key=cost)
        best_samples = samples[:Q1]
        model.update(alpha,empirical_probabilities(best_samples))


# initial: Initial puzzle with -1 for values that are NOT given
# output: Learned answer
# solution: Solution to initial
# Print the number of givens that stay the same & number of correct matches for non-givens
def summary_report(initial, output, solution):
    number_blanks = np.count_nonzero(puzzle == -1)
    number_givens = 81 - number_blanks
    number_givens_same = 0 # Number of givens that stay the same (ideally=number_givens)
    number_correct_guesses = 0 # Number of correct guesses (ideally=number_blanks)
    incorrect_cells = np.argwhere(output != solution) # Indices of cells that do not match
    for i in range(9):
        for j in range(9):
            if initial[i][j] != -1:
                if output[i][j] == initial[i][j]:
                    number_givens_same += 1
            else:
                if output[i][j] == solution[i][j]:
                    number_correct_guesses += 1
    print(f"# Blanks = {number_blanks}, # Givens = {number_givens}")
    print(f"Correct Givens: {number_givens_same} / {number_givens}")
    print(f"Correct Guesses: {number_correct_guesses} / {number_blanks}")
    print("Indices of cells that are incorrect:",incorrect_cells)


In [5]:
# Parse a file containg an unsolved sudoko puzzle and its solution, separated by newline(s)
def parse_file(file):
    with open(file) as f:
        n = f.readlines().index('\n') + 1
    puzzle = np.loadtxt(file,max_rows=n,dtype=int)
    solution = np.loadtxt(file,skiprows=n,dtype=int)
    return puzzle, solution

In [6]:
puzzle, solution = parse_file('puzzles/easy.txt')
model = Model(puzzle)

# Parameters
N = 300
Q = 100
Q1 = 10
alpha = 0.8
epsilon = 0.1
beta = 1 - epsilon

main(model, N, Q, Q1, alpha, beta)

Updating model over 300 iterations
Population Size=100,Sample Size=10,alpha=0.8,beta=0.9
Stopping condition met at iteration 19


In [7]:
initial_solution = sample(model.P0)
final_solution = sample(model.P)

cost(initial_solution), cost(final_solution)

(21, 3)

In [8]:
final_solution, solution

(array([[3, 1, 5, 4, 8, 2, 9, 6, 7],
        [4, 9, 2, 7, 6, 5, 1, 3, 8],
        [6, 7, 8, 1, 9, 3, 2, 4, 5],
        [7, 2, 3, 9, 1, 6, 5, 8, 4],
        [9, 6, 4, 2, 5, 8, 7, 1, 3],
        [5, 8, 1, 3, 7, 4, 6, 9, 2],
        [8, 5, 7, 6, 3, 1, 4, 2, 9],
        [2, 3, 8, 5, 4, 9, 8, 7, 1],
        [1, 4, 9, 8, 2, 7, 3, 5, 6]]),
 array([[3, 1, 5, 4, 8, 2, 9, 6, 7],
        [4, 9, 2, 7, 6, 5, 1, 3, 8],
        [6, 7, 8, 1, 9, 3, 2, 4, 5],
        [7, 2, 3, 9, 1, 6, 5, 8, 4],
        [9, 6, 4, 2, 5, 8, 7, 1, 3],
        [5, 8, 1, 3, 7, 4, 6, 9, 2],
        [8, 5, 7, 6, 3, 1, 4, 2, 9],
        [2, 3, 6, 5, 4, 9, 8, 7, 1],
        [1, 4, 9, 8, 2, 7, 3, 5, 6]]))

In [9]:
summary_report(model.puzzle, final_solution, solution)

# Blanks = 33, # Givens = 48
Correct Givens: 48 / 48
Correct Guesses: 32 / 33
Indices of cells that are incorrect: [[7 2]]
